In [1]:
import ujson
import re
import pandas as pd

from glob import glob
from gensim.models.word2vec import Word2Vec
from itertools import product
from IPython.display import Markdown, display
from wordfreq import top_n_list
from tqdm import tqdm_notebook
from tabulate import tabulate

In [3]:
def read_tweets(pattern, fcount=None):
    
    paths = glob(pattern)
    
    if fcount:
        paths = paths[:fcount]
    
    tweets = []
    for path in paths:
        with open(path) as fh:
            for line in fh:
                
                tweet = ujson.loads(line)
                text = re.sub('(#|@|http)\S+', '', tweet['body'])
                tokens = re.findall('[a-z0-9]+', text.lower())
                tweets.append(tokens)
                
    return tweets

In [4]:
rural = read_tweets('../../data/ma-rural.json/*.json')

In [5]:
urban = read_tweets('../../data/ma-boston.json/*.json')

In [6]:
len(rural)

2358948

In [7]:
len(urban)

4120216

In [8]:
rural_model = Word2Vec(rural, min_count=1000, workers=8)

In [9]:
urban_model = Word2Vec(urban, min_count=1000, workers=8)

In [53]:
rural_model.save('../data/models/rural-ma.bin')

In [54]:
urban_model.save('../data/models/urban-ma.bin')

In [10]:
rural_model.most_similar('earth')

[('planet', 0.6877824068069458),
 ('rise', 0.4554280638694763),
 ('ocean', 0.41596463322639465),
 ('moon', 0.4138403534889221),
 ('world', 0.4135551154613495),
 ('nature', 0.4073341190814972),
 ('spirit', 0.40715155005455017),
 ('society', 0.39870408177375793),
 ('flag', 0.38992565870285034),
 ('modern', 0.3835412263870239)]

In [11]:
urban_model.most_similar('earth')

[('planet', 0.7498504519462585),
 ('heaven', 0.4522686004638672),
 ('world', 0.45225104689598083),
 ('nature', 0.4353023171424866),
 ('ocean', 0.432361900806427),
 ('spirit', 0.4123165011405945),
 ('side', 0.397872656583786),
 ('nation', 0.39624789357185364),
 ('animals', 0.3900935649871826),
 ('universe', 0.38589024543762207)]

In [12]:
def score_shift(m1, m2, token, n=10):
    
    m1_similar = [t for t, _ in m1.most_similar(token, topn=n)]
    m2_similar = [t for t, _ in m2.most_similar(token, topn=n)]
    
    score = 0
    for t1, t2 in product(m1_similar, m2_similar):
        if t1 in m2 and t2 in m2:
            score += m2.similarity(t1, t2)
        
    return score

In [13]:
rural_vocab = set([t for t, v in rural_model.wv.vocab.items() if v.count > 1000])
urban_vocab = set([t for t, v in urban_model.wv.vocab.items() if v.count > 1000])

In [14]:
len(rural_vocab)

2312

In [15]:
len(urban_vocab)

3715

In [16]:
vocab = set.intersection(rural_vocab, urban_vocab)

In [17]:
len(vocab)

2248

In [18]:
data = []

for token in tqdm_notebook(vocab):
    try:
        score = score_shift(urban_model, rural_model, token)
        data.append((token, score))
    except: pass

HBox(children=(IntProgress(value=0, max=2248), HTML(value='')))

In [19]:
df = pd.DataFrame(data, columns=('token', 'score'))

In [38]:
pd.options.display.max_rows = 1000

In [39]:
df.sort_values('score').head(300)

,token,score
1758,mostly,-1.767361
743,ne,0.000000
2124,max,0.000000
1838,thomas,0.000000
780,kevin,0.000000
1462,alert,0.000000
911,tune,0.754736
702,fair,1.497482
1301,french,1.518481
577,trick,1.644051


In [40]:
def compare(token):
    
    u_similar = urban_model.most_similar(token)
    r_similar = rural_model.most_similar(token)
    
    u_tokens = [[t] for t, _ in u_similar]
    r_tokens = [[t] for t, _ in r_similar]
    
    display(Markdown(f'# {token}'))
    display(Markdown(f'#### Urban'))
    print(tabulate(u_tokens))
    display(Markdown(f'#### Rural'))
    print(tabulate(r_tokens))

In [52]:
for token in df.sort_values('score').head(300)['token']:
    compare(token)

# mostly

#### Urban

----------
usually
also
kinda
weird
totally
yep
completely
certain
very
racism
----------


#### Rural

--------
sunny
lo
forecast
low
clouds
65
55
snow
69
clear
--------


...

# demand

#### Urban

--------------
govt
standards
funding
economy
infrastructure
policy
protection
economic
platform
growth
--------------


#### Rural

---------
budget
value
financial
platform
costs
growth
sales
solar
increase
provide
---------
